In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from google.protobuf.json_format import MessageToDict

In [46]:
import pandas as pd

data = np.load('./dataset/fulldata.npy')
data1 = np.load('./dataset/full_data.npy')
labels = pd.read_csv('./dataset_1/label.csv', encoding='euc-kr')


In [47]:
lab = labels.values

In [48]:
data = np.concatenate((data1, data))

In [49]:
len(data)

351177

In [5]:
name = [str(i) for i in range(0,101)]
name +=  ['112','119','1000','10000',
    '가렵다','가스','가슴','가시','각목','갇히다','감금','감전','강','강남구','강동구',
    '강북구','강서구','강풍','개','거실','걸렸다','결박','경운기','경찰','경찰차','계곡',
    '계단','고속도로','고압전선','고열','고장','부러지다','골절','탈골','곰','공사장',
    '공원','놀이터','공장','관악구','광진구','교통사고','구급대','구급대원','구급차',
    '구로구','구청','구해주세요','귀','금가다','금요일','금천구','급류','기절','기절하다',
    '깔리다','끓는물','남자친구','남편','남학생','납치','낫','낯선남자','낯선사람','낯선여자',
    '내년','내일','냄새나다','노원구','논','농약','누나','누수','누전','누출','눈','다리',
    '다음','달(월)','대문앞','도둑','절도','도로','도봉구','독극물','독버섯','독사','동대문구',
    '동생','동작구','동전','두드러기생기다','뒤','뒤통수','등','딸','떨어지다','쓰러지다',
    '뜨거운물','마당','마포구','말려주세요','말벌','맹견','머리','멧돼지','목','목요일',
    '무너지다','붕괴','무릎','문틈','물','밑에','아래','바다','반점생기다','발','발가락',
    '발목','발작','방망이','밭','배','복부','배고프다','뱀','벌','범람','벼락','병원',
    '보건소','보내주세요(경찰)','보내주세요(구급차)','복통','볼','부엌','불','불나다','붕대',
    '비닐하우스','비상약','빌라','뼈','사이','산','살충제','살해','삼키다','서대문구','서랍',
    '서울시','서초구','선반','선생님','성동구','성북구','성폭행','소방관','소방차','소화기',
    '소화전','손','손가락','손목','송파구','수영장','수요일','술취한 사람','숨을안쉬다','시청',
    '신고하세요(경찰)','심장마비','아기','아이들','어린이','아내','아들','아빠','아저씨','아줌마',
    '아파트','인대','안방','알려주세요','앞','앞집','약국','약사','양천구','어깨','어제',
    '어지러움','언니','얼굴','엄마','엘리베이터','여자친구','여학생','연기','연락해주세요','열',
    '열나다','열어주세요','엽총','영등포구','옆집','옆집 아저씨','옆집 할아버지','옆집사람','옆쪽',
    '오늘','오른쪽','오른쪽-귀','오른쪽-눈','오빠','옥상','올해','왼쪽','왼쪽-귀','왼쪽-눈',
    '욕실','용산구','우리집','운동장','월요일','위','위에','위협','윗집','윗집사람','유리',
    '유치원','유치원 버스','은평구','음식물','응급대원','응급처리','의사','이마','이물질','이번',
    '이상한사람','이웃집','일요일','임산부','임신한아내','자동차','자살','자상','작년','작은방',
    '장난감','장단지','절단','제초제','조난','종로구','주','중구','중랑구','지난','지혈대','진통제',
    '질식','집','집단폭행','차밖','차안','창문','창백하다','체온계','총','추락','축사','출산',
    '출혈','피나다','친구','침수','칼','코','택시','토요일','토하다','통학버스','트랙터','트럭',
    '파도','파편','팔','팔꿈치','폭발','폭탄','폭우','폭행','학교','학생','할머니','할아버지',
    '함몰되다','해(연)','해독제','해열제','허리','허벅지','현관','현관앞','협박','형','호흡곤란',
    '호흡기','홍수','화상','화약','화요일','화장실','화재'
]

len(name)

419

In [10]:
a = data[0,0,-1]
str(int(a))

'0'

In [50]:
for i in range(len(data)):
    num = data[i,0,-1]
    label_num = lab[int(num)][0]
    num = name.index(label_num)
    data[i,:,-1] = num


In [51]:
np.save(os.path.join('dataset/data'), data)

In [12]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from google.protobuf.json_format import MessageToDict

actions = labels

full_data = np.zeros(199)

# LSTM Window Size
seq_length = 10

VIDEO_FILES = []
dir_path = './dataset_1/viedo'
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        VIDEO_FILES.append(os.path.join(root, file))

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    model_complexity=0,
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

os.makedirs('dataset', exist_ok=True)
zero = np.zeros(99) 

for idx, file in enumerate(VIDEO_FILES):
    cap = cv2.VideoCapture(file)
    action = actions[idx][0]
    data = []

    while cap.isOpened():

        ret, img = cap.read()
        if not ret:
            break

        img = img.copy()
        img = cv2.flip(img, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if result.multi_hand_landmarks:
            data_arr = []
            for res in result.multi_hand_landmarks:
                joint = np.zeros((21, 4))
                for j, lm in enumerate(res.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

                # Compute angles between joints
                v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
                v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
                v = v2 - v1 # [20, 3]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                    v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                    v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                angle = np.degrees(angle) # Convert radian to degree

                angle_label = np.array(angle, dtype=np.float32)
                # angle_label = np.append(angle_label, idx)

                d = np.concatenate([joint.flatten(), angle_label])

                mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
                data_arr.extend(d)
               
            if len(data_arr) == 99:
                handedness_dict = MessageToDict(result.multi_handedness[0])
                if handedness_dict['classification'][0]['label'] == 'Right':
                    data_arr = np.concatenate((zero, data_arr))
                else:
                    data_arr = np.concatenate((data_arr, zero))
            elif len(data_arr) > 198:
                continue
            data_arr = np.append(data_arr, idx)
            data.append(data_arr)
            
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break
    

    data = np.array(data)
    # Create sequence data
    try:
        full_seq_data = []
        for seq in range(len(data) - seq_length):
            full_seq_data.append(data[seq:seq + seq_length])
        full_seq_data = np.array(full_seq_data)

        if idx == 0:
            full_data = full_seq_data
            continue

        full_data = np.concatenate((full_data, full_seq_data))
        print(action,idx, full_data.shape)
        
    except:
        print('ERROR!!', action, idx)
        pass

cv2.destroyAllWindows()
cap.release()

np.save(os.path.join('dataset/fulldata'), full_data)

70 1 (310, 10, 199)
71 2 (495, 10, 199)
72 3 (665, 10, 199)
73 4 (835, 10, 199)
74 5 (1005, 10, 199)
75 6 (1190, 10, 199)
76 7 (1375, 10, 199)
77 8 (1545, 10, 199)
78 9 (1700, 10, 199)
79 10 (1732, 10, 199)
80 11 (1757, 10, 199)
81 12 (1794, 10, 199)
82 13 (1836, 10, 199)
83 14 (1874, 10, 199)
84 15 (1913, 10, 199)
85 16 (1951, 10, 199)
86 17 (1985, 10, 199)
87 18 (2024, 10, 199)
88 19 (2066, 10, 199)
89 20 (2108, 10, 199)
90 21 (2137, 10, 199)
91 22 (2166, 10, 199)
92 23 (2202, 10, 199)
94 24 (2233, 10, 199)
95 25 (2264, 10, 199)
96 26 (2297, 10, 199)
97 27 (2332, 10, 199)
98 28 (2366, 10, 199)
99 29 (2401, 10, 199)
100 30 (2439, 10, 199)
65 31 (2476, 10, 199)
66 32 (2515, 10, 199)
93 33 (2558, 10, 199)
112 34 (2603, 10, 199)
119 35 (2647, 10, 199)
1000 36 (2680, 10, 199)
10000 37 (2707, 10, 199)
가렵다 38 (2757, 10, 199)
가스 39 (2813, 10, 199)
가슴 40 (2848, 10, 199)
가시 41 (2897, 10, 199)
각목 42 (2958, 10, 199)
갇히다 43 (3001, 10, 199)
감금 44 (3044, 10, 199)
감전 45 (3108, 10, 199)
강 46 (3154, 1

화상 346 (19807, 10, 199)
화약 347 (19872, 10, 199)
화요일 348 (19910, 10, 199)
화장실 349 (19944, 10, 199)
화재 350 (20013, 10, 199)
장단지 351 (20092, 10, 199)
0 352 (20144, 10, 199)
1 353 (20203, 10, 199)
2 354 (20260, 10, 199)
3 355 (20314, 10, 199)
4 356 (20373, 10, 199)
5 357 (20433, 10, 199)
6 358 (20487, 10, 199)
7 359 (20551, 10, 199)
8 360 (20610, 10, 199)
9 361 (20654, 10, 199)
10 362 (20706, 10, 199)
11 363 (20768, 10, 199)
12 364 (20819, 10, 199)
13 365 (20866, 10, 199)
14 366 (20916, 10, 199)
15 367 (20963, 10, 199)
16 368 (21016, 10, 199)
17 369 (21063, 10, 199)
18 370 (21106, 10, 199)
19 371 (21151, 10, 199)
20 372 (21187, 10, 199)
21 373 (21233, 10, 199)
22 374 (21285, 10, 199)
23 375 (21335, 10, 199)
24 376 (21390, 10, 199)
25 377 (21437, 10, 199)
26 378 (21497, 10, 199)
27 379 (21551, 10, 199)
28 380 (21604, 10, 199)
29 381 (21665, 10, 199)
30 382 (21714, 10, 199)
31 383 (21768, 10, 199)
32 384 (21818, 10, 199)
33 385 (21873, 10, 199)
34 386 (21918, 10, 199)
35 387 (21962, 10, 199)

의사 681 (40094, 10, 199)
이마 682 (40160, 10, 199)
이물질 683 (40232, 10, 199)
이번 684 (40279, 10, 199)
이상한사람 685 (40359, 10, 199)
이웃집 686 (40424, 10, 199)
일요일 687 (40493, 10, 199)
임산부 688 (40562, 10, 199)
임신한아내 689 (40661, 10, 199)
자동차 690 (40731, 10, 199)
자살 691 (40799, 10, 199)
자상 692 (40877, 10, 199)
작년 693 (40931, 10, 199)
작은방 694 (40993, 10, 199)
장난감 695 (41053, 10, 199)
장단지 696 (41086, 10, 199)
절단 697 (41146, 10, 199)
제초제 698 (41236, 10, 199)
조난 699 (41305, 10, 199)
종로구 700 (41366, 10, 199)
주 701 (41417, 10, 199)
중구 702 (41482, 10, 199)
중랑구 703 (41546, 10, 199)
지난 704 (41599, 10, 199)
지혈대 705 (41705, 10, 199)
진통제 706 (41801, 10, 199)
질식 707 (41870, 10, 199)
집 708 (41908, 10, 199)
집단폭행 709 (41993, 10, 199)
차밖 710 (42065, 10, 199)
차안 711 (42129, 10, 199)
창문 712 (42220, 10, 199)
창백하다 713 (42283, 10, 199)
체온계 714 (42391, 10, 199)
총 715 (42443, 10, 199)
추락 716 (42509, 10, 199)
축사 717 (42575, 10, 199)
출산 718 (42637, 10, 199)
출혈 719 (42710, 10, 199)
친구 720 (42755, 10, 199)
침수 721 (42833, 10, 

선생님 1017 (59213, 10, 199)
성동구 1018 (59292, 10, 199)
성북구 1019 (59356, 10, 199)
성폭행 1020 (59414, 10, 199)
소방관 1021 (59491, 10, 199)
소방차 1022 (59587, 10, 199)
소화기 1023 (59667, 10, 199)
소화전 1024 (59771, 10, 199)
손 1025 (59805, 10, 199)
손목 1026 (59875, 10, 199)
송파구 1027 (59937, 10, 199)
수영장 1028 (60010, 10, 199)
수요일 1029 (60051, 10, 199)
술취한 사람 1030 (60113, 10, 199)
숨을안쉬다 1031 (60181, 10, 199)
시청 1032 (60236, 10, 199)
신고하세요(경찰) 1033 (60281, 10, 199)
심장마비 1034 (60326, 10, 199)
아기 1035 (60366, 10, 199)
아이들 1036 (60406, 10, 199)
어린이 1037 (60446, 10, 199)
아내 1038 (60482, 10, 199)
아들 1039 (60513, 10, 199)
아빠 1040 (60548, 10, 199)
아저씨 1041 (60585, 10, 199)
아줌마 1042 (60626, 10, 199)
아파트 1043 (60669, 10, 199)
인대 1044 (60729, 10, 199)
안방 1045 (60792, 10, 199)
알려주세요 1046 (60851, 10, 199)
앞 1047 (60887, 10, 199)
앞집 1048 (60953, 10, 199)
약국 1049 (61008, 10, 199)
약사 1050 (61058, 10, 199)
양천구 1051 (61125, 10, 199)
어깨 1052 (61176, 10, 199)
어제 1053 (61225, 10, 199)
어지러움 1054 (61269, 10, 199)
언니 1055 (61307

금요일 1339 (79992, 10, 199)
금천구 1340 (80056, 10, 199)
급류 1341 (80145, 10, 199)
기절 1342 (80218, 10, 199)
기절하다 1343 (80291, 10, 199)
깔리다 1344 (80364, 10, 199)
끓는물 1345 (80449, 10, 199)
남자친구 1346 (80504, 10, 199)
남편 1347 (80559, 10, 199)
남학생 1348 (80626, 10, 199)
납치 1349 (80686, 10, 199)
낫 1350 (80752, 10, 199)
낯선남자 1351 (80820, 10, 199)
낯선사람 1352 (80906, 10, 199)
낯선여자 1353 (80980, 10, 199)
내년 1354 (81044, 10, 199)
내일 1355 (81092, 10, 199)
냄새나다 1356 (81164, 10, 199)
노원구 1357 (81228, 10, 199)
논 1358 (81304, 10, 199)
농약 1359 (81402, 10, 199)
누나 1360 (81445, 10, 199)
누수 1361 (81527, 10, 199)
누전 1362 (81600, 10, 199)
누출 1363 (81680, 10, 199)
눈 1364 (81753, 10, 199)
다리 1365 (81821, 10, 199)
다음 1366 (81879, 10, 199)
달(월) 1367 (81941, 10, 199)
대문앞 1368 (82039, 10, 199)
도둑 1369 (82102, 10, 199)
절도 1370 (82165, 10, 199)
도로 1371 (82221, 10, 199)
도봉구 1372 (82295, 10, 199)
독극물 1373 (82375, 10, 199)
독버섯 1374 (82469, 10, 199)
독사 1375 (82562, 10, 199)
동대문구 1376 (82641, 10, 199)
동생 1377 (82690, 10, 199)
동작

49 1658 (103723, 10, 199)
50 1659 (103891, 10, 199)
51 1660 (104005, 10, 199)
52 1661 (104103, 10, 199)
53 1662 (104215, 10, 199)
54 1663 (104328, 10, 199)
55 1664 (104435, 10, 199)
56 1665 (104535, 10, 199)
57 1666 (104652, 10, 199)
58 1667 (104835, 10, 199)
59 1668 (104926, 10, 199)
60 1669 (105036, 10, 199)
61 1670 (105143, 10, 199)
62 1671 (105242, 10, 199)
63 1672 (105357, 10, 199)
64 1673 (105433, 10, 199)
65 1674 (105612, 10, 199)
66 1675 (105714, 10, 199)
67 1676 (105802, 10, 199)
68 1677 (105901, 10, 199)
69 1678 (105995, 10, 199)
70 1679 (106171, 10, 199)
71 1680 (106269, 10, 199)
72 1681 (106376, 10, 199)
73 1682 (106502, 10, 199)
74 1683 (106648, 10, 199)
75 1684 (106758, 10, 199)
76 1685 (106869, 10, 199)
77 1686 (106969, 10, 199)
78 1687 (107071, 10, 199)
79 1688 (107199, 10, 199)
80 1689 (107298, 10, 199)
81 1690 (107399, 10, 199)
82 1691 (107496, 10, 199)
83 1692 (107617, 10, 199)
84 1693 (107702, 10, 199)
85 1694 (107804, 10, 199)
86 1695 (107911, 10, 199)
87 1696 (108

주 1966 (133971, 10, 199)
중구 1967 (134050, 10, 199)
중랑구 1968 (134128, 10, 199)
지난 1969 (134209, 10, 199)
지혈대 1970 (134330, 10, 199)
진통제 1971 (134423, 10, 199)
질식 1972 (134542, 10, 199)
집 1973 (134642, 10, 199)
집단폭행 1974 (134737, 10, 199)
차밖 1975 (134825, 10, 199)
차안 1976 (134916, 10, 199)
창문 1977 (135002, 10, 199)
창백하다 1978 (135091, 10, 199)
체온계 1979 (135198, 10, 199)
총 1980 (135279, 10, 199)
추락 1981 (135386, 10, 199)
축사 1982 (135449, 10, 199)
출산 1983 (135534, 10, 199)
출혈 1984 (135640, 10, 199)
피나다 1985 (135746, 10, 199)
친구 1986 (135819, 10, 199)
침수 1987 (135938, 10, 199)
칼 1988 (136023, 10, 199)
코 1989 (136148, 10, 199)
택시 1990 (136240, 10, 199)
토요일 1991 (136330, 10, 199)
토하다 1992 (136415, 10, 199)
통학버스 1993 (136496, 10, 199)
트랙터 1994 (136594, 10, 199)
트럭 1995 (136675, 10, 199)
파도 1996 (136770, 10, 199)
파편 1997 (136847, 10, 199)
팔 1998 (136943, 10, 199)
팔꿈치 1999 (137044, 10, 199)
폭발 2000 (137130, 10, 199)
폭탄 2001 (137216, 10, 199)
폭우 2002 (137335, 10, 199)
폭행 2003 (137415, 10, 199)
학교 

서대문구 2277 (158520, 10, 199)
서랍 2278 (158574, 10, 199)
서울시 2279 (158642, 10, 199)
서초구 2280 (158716, 10, 199)
선반 2281 (158761, 10, 199)
선생님 2282 (158823, 10, 199)
성동구 2283 (158916, 10, 199)
성북구 2284 (159004, 10, 199)
성폭행 2285 (159070, 10, 199)
소방관 2286 (159156, 10, 199)
소방차 2287 (159272, 10, 199)
소화기 2288 (159381, 10, 199)
소화전 2289 (159505, 10, 199)
손 2290 (159577, 10, 199)
손가락 2291 (159661, 10, 199)
손목 2292 (159712, 10, 199)
송파구 2293 (159785, 10, 199)
수영장 2294 (159860, 10, 199)
수요일 2295 (159908, 10, 199)
술취한 사람 2296 (160006, 10, 199)
숨을안쉬다 2297 (160072, 10, 199)
시청 2298 (160143, 10, 199)
신고하세요(경찰) 2299 (160187, 10, 199)
심장마비 2300 (160245, 10, 199)
아기 2301 (160316, 10, 199)
아이들 2302 (160387, 10, 199)
어린이 2303 (160458, 10, 199)
아내 2304 (160524, 10, 199)
아들 2305 (160577, 10, 199)
아빠 2306 (160636, 10, 199)
아저씨 2307 (160696, 10, 199)
아줌마 2308 (160744, 10, 199)
아파트 2309 (160801, 10, 199)
인대 2310 (160886, 10, 199)
안방 2311 (160967, 10, 199)
알려주세요 2312 (161024, 10, 199)
앞 2313 (161072, 10, 199)


구급차 2587 (181516, 10, 199)
구로구 2588 (181592, 10, 199)
구청 2589 (181665, 10, 199)
구해주세요 2590 (181756, 10, 199)
귀 2591 (181838, 10, 199)
금가다 2592 (181910, 10, 199)
금요일 2593 (181973, 10, 199)
금천구 2594 (182056, 10, 199)
급류 2595 (182132, 10, 199)
기절 2596 (182225, 10, 199)
기절하다 2597 (182318, 10, 199)
쓰러지다 2598 (182411, 10, 199)
깔리다 2599 (182490, 10, 199)
끓는물 2600 (182576, 10, 199)
남자친구 2601 (182663, 10, 199)
남편 2602 (182731, 10, 199)
남학생 2603 (182816, 10, 199)
납치 2604 (182884, 10, 199)
낫 2605 (182983, 10, 199)
낯선남자 2606 (183080, 10, 199)
낯선사람 2607 (183175, 10, 199)
낯선여자 2608 (183254, 10, 199)
내년 2609 (183328, 10, 199)
내일 2610 (183397, 10, 199)
냄새나다 2611 (183458, 10, 199)
노원구 2612 (183535, 10, 199)
논 2613 (183611, 10, 199)
농약 2614 (183710, 10, 199)
누나 2615 (183771, 10, 199)
누수 2616 (183863, 10, 199)
누전 2617 (183955, 10, 199)
누출 2618 (184054, 10, 199)
눈 2619 (184111, 10, 199)
대문앞 2620 (184209, 10, 199)
다리 2621 (184288, 10, 199)
다음 2622 (184353, 10, 199)
달(월) 2623 (184409, 10, 199)
도둑 2624 (1844

30 2896 (201885, 10, 199)
31 2897 (201927, 10, 199)
32 2898 (201961, 10, 199)
33 2899 (202003, 10, 199)
34 2900 (202051, 10, 199)
35 2901 (202095, 10, 199)
36 2902 (202137, 10, 199)
37 2903 (202174, 10, 199)
38 2904 (202217, 10, 199)
39 2905 (202260, 10, 199)
40 2906 (202299, 10, 199)
41 2907 (202341, 10, 199)
42 2908 (202377, 10, 199)
43 2909 (202420, 10, 199)
44 2910 (202461, 10, 199)
45 2911 (202503, 10, 199)
46 2912 (202543, 10, 199)
47 2913 (202577, 10, 199)
48 2914 (202617, 10, 199)
49 2915 (202661, 10, 199)
50 2916 (202686, 10, 199)
51 2917 (202721, 10, 199)
52 2918 (202750, 10, 199)
53 2919 (202779, 10, 199)
54 2920 (202809, 10, 199)
55 2921 (202844, 10, 199)
56 2922 (202877, 10, 199)
57 2923 (202915, 10, 199)
58 2924 (202949, 10, 199)
59 2925 (202988, 10, 199)
60 2926 (203021, 10, 199)
61 2927 (203051, 10, 199)
62 2928 (203086, 10, 199)
63 2929 (203124, 10, 199)
64 2930 (203161, 10, 199)
65 2931 (203186, 10, 199)
66 2932 (203214, 10, 199)
67 2933 (203241, 10, 199)
68 2934 (203

In [3]:
np.load('dataset/fulldata.npy').shape

(206405, 10, 199)

In [3]:
data = np.load('./dataset/data.npy')

In [6]:
import numpy as np

actions = name

data.shape

(351177, 10, 199)

In [7]:
len(actions)

419

In [8]:
x = data[:,:,:-1]
y = data[:,0,-1]

x.shape, y.shape

((351177, 10, 198), (351177,))

In [10]:
from tensorflow.keras.utils import to_categorical

y = to_categorical(y)
y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
len(y[0])

419

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y)


In [13]:
x_train.shape

(280941, 10, 198)

In [14]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow

In [62]:
clear_session()

model = Sequential()
model.add(LSTM(1024, input_shape=(10, 198)))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(y.shape[-1], activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1024)              5009408   
                                                                 
 dense (Dense)               (None, 512)               524800    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 419)               54051     
                                                                 
Total params: 5,752,483
Trainable params: 5,752,483
Non-trainable params: 0
_________________________________________________________________


In [65]:
import datetime

t = datetime.datetime.now()
t = str(t)[5:10] +'_' +str(t)[11:16]
t

'04-21_10:54'

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1, min_delta=0)
mc = ModelCheckpoint(f'model_{t}.h5', monitor='val_loss', save_best_only=True, verbose=1)


hist = model.fit(x_train, y_train, validation_split=0.2, epochs=100, callbacks=[es, mc], verbose=1, batch_size=512)

Epoch 1/100
439/439 [==============================] - ETA: 0s - loss: 4.9073 - accuracy: 0.0850
Epoch 1: val_loss improved from inf to 4.13541, saving model to model_04-21_10:54.h5
439/439 [==============================] - 844s 2s/step - loss: 4.9073 - accuracy: 0.0850 - val_loss: 4.1354 - val_accuracy: 0.1571
Epoch 2/100
439/439 [==============================] - ETA: 0s - loss: 3.6004 - accuracy: 0.2226
Epoch 2: val_loss improved from 4.13541 to 3.25966, saving model to model_04-21_10:54.h5
439/439 [==============================] - 839s 2s/step - loss: 3.6004 - accuracy: 0.2226 - val_loss: 3.2597 - val_accuracy: 0.2743
Epoch 3/100
439/439 [==============================] - ETA: 0s - loss: 2.8133 - accuracy: 0.3422
Epoch 3: val_loss improved from 3.25966 to 2.64842, saving model to model_04-21_10:54.h5
439/439 [==============================] - 843s 2s/step - loss: 2.8133 - accuracy: 0.3422 - val_loss: 2.6484 - val_accuracy: 0.3730
Epoch 4/100
439/439 [=============================

In [50]:
model.save('model_0421_1054.h5')

In [42]:
y_pred = model.predict(x_test)

In [43]:
from sklearn.metrics import accuracy_score

accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))



0.8318383734836836

In [12]:
model = tensorflow.keras.models.load_model('model.h5')

In [61]:
# test

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


actions = name
seq_length = 10

# model = load_model('models/model2_1.0.h5')

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        hand_arr = []
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            d = np.concatenate([joint.flatten(), angle])
            hand_arr.extend(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)
            
        if len(hand_arr) == 99:
                hand_arr.extend(np.zeros(99))
        
        if len(hand_arr) > 198:
            continue
            
        seq.append(hand_arr)
        
        if len(seq) < seq_length:
            continue
            
        input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)
        y_pred = model.predict(input_data).squeeze()
        i_pred = int(np.argmax(y_pred))
        conf = y_pred[i_pred]
        if conf < 0.8:
            continue
            
        print(name[i_pred])
        action = actions[i_pred]
        action_seq.append(action)
        if len(action_seq) < 3:
            continue
        this_action = '?'
        if action_seq[-1] == action_seq[-2] == action_seq[-3]:
            this_action = action
        font = ImageFont.truetype("fonts/gulim.ttc", 20)
        img = Image.fromarray(img)
        draw = ImageDraw.Draw(img)
        draw.text((30,50), this_action, font=font, fill=(0,0,255))
        img = np.array(img)

    # out.write(img0)
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

            
cv2.destroyAllWindows()
cap.release()